In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from utils import log_wrapper_for_logged_data, DICT_RC_PARAM

In [ ]:
df = pd.read_csv(f"../generator/data/cplex/final_results/transition_to_ER/L21_ER.csv")

In [ ]:
df = df[8:]
df = df.astype(float)
df

In [ ]:
data = np.log10(df["Process TTS"])[:500]

med_val = np.median(data)
std_val = np.std(data)
nb_ER_runs = len(df)

med_val, std_val, nb_ER_runs

In [ ]:
a = plt.boxplot(data)
min_wiskers, max_whiskers = [item.get_ydata()[1] for item in a["whiskers"]]
Q1 = data.describe()["25%"]
Q3 = data.describe()["75%"]

In [ ]:
### the transition

In [ ]:
df_rewired = pd.read_csv(
    f"../generator/data/cplex/final_results/transition_to_ER/L21_rewired_final.csv"
)

In [ ]:
max_seed = df_rewired.groupby("Frac Rewired").count()["L"][20]
max_seed

In [ ]:
max_seed = 150

df_regular = pd.read_csv(f"../generator/data/cplex/run_time_d0.8_UDG_8vCPU.csv")
df_regular = df_regular[df_regular["L"] == 21].set_index("Seed")[:max_seed]
df_regular["Frac Rewired"] = 0

In [ ]:
# df = df_rewired
df = pd.concat([df_rewired, df_regular])
df["log10 Process TTS"] = np.log10(df["Process TTS"])
df["Fraction Rewired"] = np.round(df["Frac Rewired"] * 0.05, 2)

In [ ]:
plt.rcParams.update(DICT_RC_PARAM)
fig, ax = plt.subplots(figsize=(8, 8))

positions = np.unique(df["Fraction Rewired"])
df.boxplot(
    "log10 Process TTS",
    by="Fraction Rewired",
    ax=ax,
    positions=positions,
    widths=0.02,
    color=dict(boxes="b", whiskers="b", medians="b", caps="b"),
)
ax.hlines(
    med_val, *ax.get_xlim(), color="r", label=f"median TTS ER Graph {nb_ER_runs} seeds"
)
ax.fill_between(ax.get_xlim(), Q1, Q3, facecolor="r", alpha=0.07)
ax.fill_between(ax.get_xlim(), min_wiskers, max_whiskers, facecolor="r", alpha=0.04)
# ax.set_title("Rewiring an LxL UD Union Jack graph leads to a complexity plateau for gurobi (L=21)\n 20 seeds per percentage of moved edges, constant number of edges")
ax.set_title("")
fig.suptitle("")
ax.set_xlabel("Fraction of Edges Rewired")
ax.set_ylabel("TTS (seconds)")
# ax.set_xticks(np.linspace(0,1,21))
# ax.set_xlim([0,1.05])
log_wrapper_for_logged_data(ax, y_axis=True, base=10)
ax.set_xlim([-0.05, 1.05])
ax.set_xticks([np.round(r, 2) for r in np.linspace(0, 1, 11)])
ax.set_xticklabels([np.round(r, 2) for r in np.linspace(0, 1, 11)])
plt.savefig("Process_TTS_rewiring.png")